In [18]:
##pip install jupyter-dash
#conda install -c conda-forge -c plotly jupyter-dash

In [6]:
import plotly.express as px
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
#app.run_server(mode='inline')
app.run_server(mode='external')
#app.run_server(mode='jupyterlab')

Dash app running on http://127.0.0.1:8050/


In [9]:
# ERROR REPORTING
# ansi2html library
# JupyterDash.infer_jupyter_proxy_config()

# Dash Enterprise Workspaces.

In [16]:
import warnings
warnings.filterwarnings("ignore")

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash import Dash
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

#import plotly.plotly as py
import chart_studio.plotly as py
# pip install chart_studio
from plotly.graph_objs import *

app = JupyterDash(__name__)
server = app.server
app.title='Iris Data Dashboard'

# do all the machine learning
iris = datasets.load_iris()
colors = ["#E41A1C", "#377EB8", "#4DAF4A", \
          "#984EA3", "#FF7F00", "#FFFF33", \
          "#A65628", "#F781BF", "#999999"]
number_of_clusters = range(9)


df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
col = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
X = df[col]
y = df["target"]

df_list = []
# declaring our algorithm and its parameters
for n in number_of_clusters:
    kmeans = KMeans(n_clusters=n + 1)

    # fit data to features
    kmeans.fit(X)

    results_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    results_df["predicted_classes"] = kmeans.labels_

    num_of_clusters = results_df['predicted_classes'].nunique()
    df_list.append({
        'results_df': results_df,
        'num_of_clusters': num_of_clusters
    })

dropdown_val = iris.feature_names

app.layout = html.Div(className="container", children=[
                html.Div(className="jumbotron text-center", children=[
                    html.H1("Iris Analysis"),
                    html.P("Select the X and Y to visualize the data using chosen number of clusters"),
                    html.P("A simple classification algorithm using K Nearest Neighbors")
                    ]),
                dcc.Dropdown(className="col-md-4", id="dropdown_x",
                    options=[
                        {'label': val, 'value': val} for val in dropdown_val
                    ],
                    value=dropdown_val[0] #default value for first launch
                ),
                dcc.Dropdown(className="col-md-4", id="dropdown_y",
                    options=[
                        {'label': val, 'value': val} for val in dropdown_val
                    ],
                    value=dropdown_val[1]
                ),
                dcc.Dropdown(className="col-md-4", id="dropdown_k",
                    options=[
                        {'label': str(val+1), 'value': val} for val in number_of_clusters
                    ],
                    value=number_of_clusters[3]
                ),
                html.Div(style={"padding": "20px"}, children=[
                    dcc.Graph(id="cluster")
                ])
    ])

# import external css
app.css.append_css({"external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css"})

# import external javascript
app.scripts.append_script({"external_url": "https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js"})

# insert callbacks
@app.callback(Output("cluster", "figure"), [Input('dropdown_x', 'value'),Input('dropdown_y', 'value'),Input('dropdown_k', 'value')])
def update_graph(x, y, k):
    data = []
    num_of_clusters = df_list[k]['num_of_clusters']
    for i in range(num_of_clusters):
        results_df = df_list[k]['results_df']
        cluster_df = results_df[results_df["predicted_classes"] == i]
        data.append({
            "x": cluster_df[x],
            "y": cluster_df[y],
            "type": "scatter",
            "mode": "markers",
            "name": f"class_{i}",
            "marker": dict(
                color = colors[i],
                size = 10
            )
        })
        
    layout = {
        "hovermode": "closest", 
        "margin": {
            "r": 10, 
            "t": 25, 
            "b": 100, 
            "l": 60
        }, 
        "title": f"Iris Dataset - {x} vs {y}", 
        "xaxis": {
            "domain": [0, 1], 
            "title": x
        }, 
        "yaxis": {
            "domain": [0, 1], 
            "title": y
        }
    }

    fig = Figure(data=data, layout=layout)
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline')

In [17]:
from sklearn import datasets
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
import ipywidgets as widgets
from IPython.display import display, clear_output
output_notebook()
 
"""Load Iris dataset and transform the pandas DataFrame"""
iris = datasets.load_iris()
data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])
 
"""Define callback function for the UI"""
def var_dropdown(x):
    """This function is executed when a dropdown value is changed.
    It creates a new figure according to the new dropdown values."""
    p = create_figure(
    x_dropdown.children[0].value,
    y_dropdown.children[0].value,
    data)
    fig[0] = p
     
    for species, checkbox in species_checkboxes.items():
        check = checkbox.children[0].value
        fig[0].select_one({'name': species}).visible = check
     
    with output_figure:
        clear_output(True)
        show(fig[0])
    fig[0]=p
     
    return x
 
def f_species_checkbox(x, q):
    """This function is executed when a checkbox is clicked.
    It directly changes the visibility of the current figure."""
    fig[0].select_one({'name': q}).visible = x
    with output_figure:
        clear_output(True)
        show(fig[0])
    return x
 
def create_figure(x_var, y_var, data):
    """This is a helper function that creates a new figure and 
    plots values from all three species. x_var and y_var control
    the features on each axis."""
    species_colors=['coral', 'deepskyblue', 'darkblue']
    p = figure(title="",
               x_axis_label=x_var,
               y_axis_label=y_var)
    species_nr = 0
    for species in iris['target_names']:
        curr_dtps = data['target'] == species_nr
        circle = p.circle(
            data[x_var][curr_dtps],
            data[y_var][curr_dtps],
            line_width=2,
            color=species_colors[species_nr],
            name=species
            )
        species_nr += 1
    return p
 
# The output widget is where we direct our figures
output_figure = widgets.Output()
 
# Create the default figure
fig = []  # Storing the figure in a singular list is a bit of a 
          # hack. We need it to properly mutate the current
          # figure in our callbacks.
p = create_figure(
    iris['feature_names'][0],
    iris['feature_names'][1],
    data)
fig.append(p)
with output_figure:
    show(fig[0])
 
# Checkboxes to select visible species.
species_checkboxes = {}
for species in iris['target_names']:
    curr_cb = widgets.interactive(f_species_checkbox,
                                  x=True,
                                  q=widgets.fixed(species))
    curr_cb.children[0].description = species
    species_checkboxes[species] = curr_cb
     
"""Create the widgets in the menu"""
# Dropdown menu for x-axis feature.
x_dropdown = widgets.interactive(var_dropdown,
                                 x=iris['feature_names']);
x_dropdown.children[0].description = 'x-axis'
x_dropdown.children[0].value = iris['feature_names'][0]
 
# Dropdown menu for y-axis feature.
y_dropdown = widgets.interactive(var_dropdown,
                                 x=iris['feature_names']);
y_dropdown.children[0].description = 'y-axis'
y_dropdown.children[0].value = iris['feature_names'][1]
 
 
 
# This creates the menu 
menu=widgets.VBox([x_dropdown,
                   y_dropdown,
                   *species_checkboxes.values()])
 
"""Create the full app with menu and output"""
# The Layout adds some styling to our app.
# You can add Layout to any widget.
app_layout = widgets.Layout(display='flex',
                flex_flow='row nowrap',
                align_items='center',
                border='none',
                width='100%',
                margin='5px 5px 5px 5px')
 
# The final app is just a box
app=widgets.Box([menu, output_figure], layout=app_layout)
 
# Display the app
display(app)


Loading BokehJS ...

Box(children=(VBox(children=(interactive(children=(Dropdown(description='x-axis', options=('sepal length (cm)'…